# Loan Denial discrimination analysis

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [107]:
lc_accepted=pd.read_csv('LC_Data/Lending_Club_Accepted_2014_2018.csv')
lc_rejected=pd.read_csv('LC_Data/Lending_Club_Rejected_2014_2018.csv')
lc_common=[ 'loan_amnt', 'issue_d', 'purpose', 'dti', 'addr_state', 'emp_length', 'policy_code']
lc_accepted=lc_accepted[lc_common]
lc_accepted.head()

/var/folders/d3/ry6v0r7j28d4wdqwrz_hvdqc0000gn/T/ipykernel_49740/1814094668.py:1: DtypeWarning: Columns (19,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  lc_accepted=pd.read_csv('LC_Data/Lending_Club_Accepted_2014_2018.csv')


,loan_amnt,issue_d,purpose,dti,addr_state,emp_length,policy_code
0,3600.0,Dec-2015,debt_consolidation,5.91,PA,10+ years,1.0
1,24700.0,Dec-2015,small_business,16.06,SD,10+ years,1.0
2,20000.0,Dec-2015,home_improvement,10.78,IL,10+ years,1.0
3,35000.0,Dec-2015,debt_consolidation,17.06,NJ,10+ years,1.0
4,10400.0,Dec-2015,major_purchase,25.37,PA,3 years,1.0


In [108]:
lc_rejected.drop(['Zip_Code', 'Risk_Score'], axis=1, inplace=True)
lc_rejected.head()
print(lc_rejected['Debt_to_Income_Ratio'])

0            2.69%
1           28.26%
2              -1%
3           21.43%
4            8.49%
             ...  
26132303    41.26%
26132304     1.48%
26132305    10.26%
26132306    17.71%
26132307    10.58%
Name: Debt_to_Income_Ratio, Length: 26132308, dtype: object


In [109]:
rename_dict={'addr_state':'State', 'loan_amnt':'Amount_Requested', 'issue_d':'Application_Date', 'purpose':'Loan_Title', 'dti':'Debt_to_Income_Ratio', 'emp_length':'Employment_Length', 'policy_code':'Policy_Code'}
lc_accepted.rename(columns=rename_dict, inplace=True)
lc_accepted.head()

/var/folders/d3/ry6v0r7j28d4wdqwrz_hvdqc0000gn/T/ipykernel_49740/358241859.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lc_accepted.rename(columns=rename_dict, inplace=True)


,Amount_Requested,Application_Date,Loan_Title,Debt_to_Income_Ratio,State,Employment_Length,Policy_Code
0,3600.0,Dec-2015,debt_consolidation,5.91,PA,10+ years,1.0
1,24700.0,Dec-2015,small_business,16.06,SD,10+ years,1.0
2,20000.0,Dec-2015,home_improvement,10.78,IL,10+ years,1.0
3,35000.0,Dec-2015,debt_consolidation,17.06,NJ,10+ years,1.0
4,10400.0,Dec-2015,major_purchase,25.37,PA,3 years,1.0


In [110]:
#create column to indicate loans were accepted or rejected
lc_accepted['Accepted']=1
lc_accepted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029952 entries, 0 to 2029951
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount_Requested      float64
 1   Application_Date      object 
 2   Loan_Title            object 
 3   Debt_to_Income_Ratio  float64
 4   State                 object 
 5   Employment_Length     object 
 6   Policy_Code           float64
 7   Accepted              int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 123.9+ MB


In [111]:
lc_rejected['Accepted']=0
lc_rejected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26132308 entries, 0 to 26132307
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount_Requested      float64
 1   Application_Date      object 
 2   Loan_Title            object 
 3   Debt_to_Income_Ratio  object 
 4   State                 object 
 5   Employment_Length     object 
 6   Policy_Code           float64
 7   Accepted              int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 1.6+ GB


In [112]:
#convert Debt_to_Income_Ratio to numeric in lc_rejected
lc_rejected['Debt_to_Income_Ratio']=pd.to_numeric(lc_rejected['Debt_to_Income_Ratio'].str.strip('%'))
lc_rejected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26132308 entries, 0 to 26132307
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount_Requested      float64
 1   Application_Date      object 
 2   Loan_Title            object 
 3   Debt_to_Income_Ratio  float64
 4   State                 object 
 5   Employment_Length     object 
 6   Policy_Code           float64
 7   Accepted              int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 1.6+ GB


In [113]:
#append lc_accepted and lc_rejected to create a single dataframe
lc_all=lc_accepted.append(lc_rejected, ignore_index=True)
lc_all['Employment_Length'].fillna(0, inplace=True)
lc_all.dropna(inplace=True)
lc_all.info()

/var/folders/d3/ry6v0r7j28d4wdqwrz_hvdqc0000gn/T/ipykernel_49740/2391448617.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lc_all=lc_accepted.append(lc_rejected, ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28158341 entries, 0 to 28162259
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount_Requested      float64
 1   Application_Date      object 
 2   Loan_Title            object 
 3   Debt_to_Income_Ratio  float64
 4   State                 object 
 5   Employment_Length     object 
 6   Policy_Code           float64
 7   Accepted              int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 1.9+ GB


In [114]:
lc_all.head()

,Amount_Requested,Application_Date,Loan_Title,Debt_to_Income_Ratio,State,Employment_Length,Policy_Code,Accepted
0,3600.0,Dec-2015,debt_consolidation,5.91,PA,10+ years,1.0,1
1,24700.0,Dec-2015,small_business,16.06,SD,10+ years,1.0,1
2,20000.0,Dec-2015,home_improvement,10.78,IL,10+ years,1.0,1
3,35000.0,Dec-2015,debt_consolidation,17.06,NJ,10+ years,1.0,1
4,10400.0,Dec-2015,major_purchase,25.37,PA,3 years,1.0,1


In [115]:
#sort lc_all by state
lc_all.sort_values(by='State', inplace=True)
lc_all.head(1000)

,Amount_Requested,Application_Date,Loan_Title,Debt_to_Income_Ratio,State,Employment_Length,Policy_Code,Accepted
8799113,30000.0,2014-05-14,credit_card,21.16,AK,< 1 year,0.0,0
22800799,2000.0,2015-04-01,other,26.10,AK,5 years,0.0,0
23170119,10000.0,2015-05-21,debt_consolidation,20.76,AK,< 1 year,0.0,0
20206634,35000.0,2018-06-30,Credit card refinancing,86.45,AK,< 1 year,0.0,0
8744231,2000.0,2014-05-02,debt_consolidation,21.84,AK,< 1 year,0.0,0
...,...,...,...,...,...,...,...,...
11876147,20000.0,2018-03-22,Debt consolidation,17.48,AK,< 1 year,0.0,0
18505251,10000.0,2018-04-30,Debt consolidation,17.50,AK,< 1 year,0.0,0
12019240,3500.0,2018-03-28,Home improvement,100.00,AK,< 1 year,0.0,0
9539007,1000.0,2014-09-07,other,-1.00,AK,< 1 year,0.0,0


## Add the categorization of the states to the dataframe

In [116]:
#import the state categories from the csv file
devscaled=pd.read_csv('devscaled.csv')
devscaled.set_index('State', inplace=True)
devscaled.head()

,2014_gdp,2015_gdp,2016_gdp,2017_gdp,2018_gdp,2014_unemp,2015_unemp,2016_unemp,2017_unemp,2018_unemp,...,2014_rank,2015_rank,2016_rank,2017_rank,2018_rank,2014,2015,2016,2017,2018
State,,,,,,,,,,,,,,,,,,,,,
AL,0.072278,0.069670,0.068742,0.066898,0.065489,-0.788462,-0.804878,-0.725,-0.434783,-0.357143,...,41.0,43.0,44.0,42.0,44.0,4,4,5,4,5
AK,0.011349,0.011004,0.010356,0.009803,0.009041,-0.807692,-0.902439,-1.000,-1.000000,-1.000000,...,51.0,51.0,51.0,51.0,51.0,5,5,5,5,5
AZ,0.111370,0.108858,0.109180,0.108649,0.108648,-0.788462,-0.804878,-0.625,-0.543478,-0.571429,...,48.0,46.0,46.0,48.0,48.0,5,5,5,5,5
AR,0.037188,0.035747,0.034768,0.033589,0.033023,-0.634615,-0.536585,-0.275,-0.282609,-0.309524,...,39.0,36.0,28.0,31.0,36.0,4,4,3,3,4
CA,1.000000,1.000000,1.000000,1.000000,1.000000,-0.923077,-0.829268,-0.650,-0.521739,-0.428571,...,5.0,2.0,2.0,1.0,1.0,1,1,1,1,1


In [117]:
devscaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, AL to WY
Data columns (total 40 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   2014_gdp     51 non-null     float64
 1   2015_gdp     51 non-null     float64
 2   2016_gdp     51 non-null     float64
 3   2017_gdp     51 non-null     float64
 4   2018_gdp     51 non-null     float64
 5   2014_unemp   51 non-null     float64
 6   2015_unemp   51 non-null     float64
 7   2016_unemp   51 non-null     float64
 8   2017_unemp   51 non-null     float64
 9   2018_unemp   51 non-null     float64
 10  2018_health  51 non-null     float64
 11  2017_health  51 non-null     float64
 12  2016_health  51 non-null     float64
 13  2015_health  51 non-null     float64
 14  2014_health  51 non-null     float64
 15  2018_edu     51 non-null     float64
 16  2017_edu     51 non-null     float64
 17  2016_edu     51 non-null     float64
 18  2015_edu     51 non-null     float64
 19  2014_edu     5

In [118]:
lc_all['Year']=lc_all['Application_Date'].str.extract('(\d{4})', expand=False)
lc_all.head()

,Amount_Requested,Application_Date,Loan_Title,Debt_to_Income_Ratio,State,Employment_Length,Policy_Code,Accepted,Year
8799113,30000.0,2014-05-14,credit_card,21.16,AK,< 1 year,0.0,0,2014
22800799,2000.0,2015-04-01,other,26.10,AK,5 years,0.0,0,2015
23170119,10000.0,2015-05-21,debt_consolidation,20.76,AK,< 1 year,0.0,0,2015
20206634,35000.0,2018-06-30,Credit card refinancing,86.45,AK,< 1 year,0.0,0,2018
8744231,2000.0,2014-05-02,debt_consolidation,21.84,AK,< 1 year,0.0,0,2014


In [119]:
#add a State_Category column to the lc_all dataframe by the year and the state in the devscaled dataframe
lc_all['State_Category']=lc_all.apply(lambda x: devscaled.loc[x['State'], x['Year']], axis=1)



In [120]:
lc_all.head(10000)

,Amount_Requested,Application_Date,Loan_Title,Debt_to_Income_Ratio,State,Employment_Length,Policy_Code,Accepted,Year,State_Category
8799113,30000.0,2014-05-14,credit_card,21.16,AK,< 1 year,0.0,0,2014,5
22800799,2000.0,2015-04-01,other,26.10,AK,5 years,0.0,0,2015,5
23170119,10000.0,2015-05-21,debt_consolidation,20.76,AK,< 1 year,0.0,0,2015,5
20206634,35000.0,2018-06-30,Credit card refinancing,86.45,AK,< 1 year,0.0,0,2018,5
8744231,2000.0,2014-05-02,debt_consolidation,21.84,AK,< 1 year,0.0,0,2014,5
...,...,...,...,...,...,...,...,...,...,...
11021735,25000.0,2018-02-08,Debt consolidation,15.86,AK,2 years,0.0,0,2018,5
8156784,23000.0,2018-09-26,Debt consolidation,23.00,AK,< 1 year,0.0,0,2018,5
11067661,5000.0,2018-02-12,Car financing,15.97,AK,< 1 year,0.0,0,2018,5
5517298,5000.0,2017-03-21,debt_consolidation,6.27,AK,< 1 year,0.0,0,2017,5
